In [1]:
import typesense
import os
from neo4j import GraphDatabase
from pathlib import Path
from dotenv import load_dotenv

In [2]:
load_dotenv()

TYPESENSE_KEY = os.environ['typesense_key']
neo4j_uri = os.environ['neo4j_uri']
neo4j_username = os.environ['neo4j_username']
neo4j_password = os.environ['neo4j_password']
neo4j_dbname = os.environ['neo4j_dbname']

In [3]:
class Neo4jGraph:

    def __init__(self, neo4j_uri:str, neo4j_username:str, neo4j_password:str, db:str)->None:
        self.uri  = neo4j_uri
        self.auth = (neo4j_username, neo4j_password)
        self.db = db
        self.driver = GraphDatabase.driver(self.uri, auth=self.auth)

    def query(self, query:str, params:dict={}):
        with self.driver.session(database=self.db) as session:
            result = session.run(query, params)
            return [r for r in result]

In [4]:
graph = Neo4jGraph(
    neo4j_uri,
    neo4j_username,
    neo4j_password,
    neo4j_dbname,
)

In [5]:
client = typesense.Client({
  'nodes': [{
    'host': 'localhost',
    'port': '8108',
    'protocol': 'http'
  }],
  'api_key': TYPESENSE_KEY,
  'connection_timeout_seconds': 2
})

In [10]:
label = "Studio"
load_script_path = Path(f"cypher-query/{label}.cypher")
BATCH_SIZE = 10000
SEARCH_INDEX_NAME = 'entity'

In [7]:
res = graph.query(f"MATCH (n:{label}) RETURN COUNT(n) as count_")
entity_count = res[0].get('count_')
print(entity_count)

with open(str(load_script_path), 'r') as fp:
    cypher_load = fp.read()

338


In [11]:
for skip_count in range(0, entity_count, BATCH_SIZE):
    res = graph.query(cypher_load, {'BATCH_SIZE':BATCH_SIZE, 'skip_count': skip_count})
    index_input_data = [doc.data() for doc in res]
    index_res = client.collections[SEARCH_INDEX_NAME].documents.import_(index_input_data, {'action': 'upsert'})